# ETF轮动策略研究报告

## 研究目标
1. 分析行业ETF、宽基ETF、主题ETF的历史表现
2. 实现并回测动量轮动、均值回归轮动、多因子轮动策略
3. 对比不同调仓周期的策略表现
4. 考虑交易成本对策略的影响

## 数据说明
- 数据来源: Tushare数据库
- 行业数据: 申万一级行业指数（27个行业）
- 宽基指数: 上证指数、沪深300、中证500等8个主要指数
- 数据范围: 2022年1月 - 2026年1月

In [ ]:
# 导入必要的库
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['PingFang HK', 'Heiti TC', 'STHeiti', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 8)

# 数据库路径
DB_PATH = '/Users/allen/workspace/python/stock/Tushare-DuckDB/tushare.db'

## 一、数据加载与预处理

In [ ]:
# 连接数据库
conn = duckdb.connect(DB_PATH, read_only=True)

# 申万一级行业代码映射
SW_L1_INDUSTRIES = {
    '801010.SI': '农林牧渔', '801030.SI': '基础化工', '801040.SI': '钢铁',
    '801050.SI': '有色金属', '801080.SI': '电子', '801110.SI': '家用电器',
    '801120.SI': '食品饮料', '801130.SI': '纺织服饰', '801140.SI': '轻工制造',
    '801150.SI': '医药生物', '801160.SI': '公用事业', '801170.SI': '交通运输',
    '801180.SI': '房地产', '801200.SI': '商贸零售', '801210.SI': '社会服务',
    '801230.SI': '综合', '801710.SI': '建筑材料', '801720.SI': '建筑装饰',
    '801730.SI': '电力设备', '801740.SI': '国防军工', '801750.SI': '计算机',
    '801760.SI': '传媒', '801770.SI': '通信', '801780.SI': '银行',
    '801790.SI': '非银金融', '801880.SI': '汽车', '801890.SI': '机械设备',
}

# 宽基指数代码映射
BROAD_INDICES = {
    '000001.SH': '上证指数', '000016.SH': '上证50', '000300.SH': '沪深300',
    '000905.SH': '中证500', '000852.SH': '中证1000', '399001.SZ': '深证成指',
    '399006.SZ': '创业板指', '000688.SH': '科创50',
}

In [ ]:
# 加载申万行业数据
industry_codes = list(SW_L1_INDUSTRIES.keys())
industry_codes_str = "', '".join(industry_codes)

industry_query = f"""
SELECT ts_code, trade_date, close, pct_change, vol, amount, pe, pb, float_mv, total_mv
FROM sw_daily
WHERE ts_code IN ('{industry_codes_str}')
  AND trade_date >= '20220101'
  AND trade_date <= '20260130'
ORDER BY ts_code, trade_date
"""
industry_data = conn.execute(industry_query).fetchdf()
industry_data['trade_date'] = pd.to_datetime(industry_data['trade_date'])
industry_data['name'] = industry_data['ts_code'].map(SW_L1_INDUSTRIES)

print(f"行业数据: {len(industry_data)} 条记录, {industry_data['ts_code'].nunique()} 个行业")
industry_data.head()

In [ ]:
# 加载宽基指数数据
broad_codes = list(BROAD_INDICES.keys())
broad_codes_str = "', '".join(broad_codes)

broad_query = f"""
SELECT ts_code, trade_date, close, pct_chg, vol, amount
FROM index_daily
WHERE ts_code IN ('{broad_codes_str}')
  AND trade_date >= '20220101'
  AND trade_date <= '20260130'
ORDER BY ts_code, trade_date
"""
broad_data = conn.execute(broad_query).fetchdf()
broad_data['trade_date'] = pd.to_datetime(broad_data['trade_date'])
broad_data['name'] = broad_data['ts_code'].map(BROAD_INDICES)

print(f"宽基指数数据: {len(broad_data)} 条记录, {broad_data['ts_code'].nunique()} 个指数")
broad_data.head()

## 二、行业ETF分析

In [ ]:
# 计算各行业的表现指标
industry_stats = []

for code, name in SW_L1_INDUSTRIES.items():
    data = industry_data[industry_data['ts_code'] == code].copy()
    if len(data) < 100:
        continue
    
    data = data.sort_values('trade_date')
    
    # 计算收益率
    total_return = data['close'].iloc[-1] / data['close'].iloc[0] - 1
    
    # 计算年化波动率
    returns = data['close'].pct_change().dropna()
    volatility = returns.std() * np.sqrt(252)
    
    # 计算最大回撤
    cummax = data['close'].cummax()
    drawdown = (data['close'] - cummax) / cummax
    max_drawdown = drawdown.min()
    
    # 夏普比率
    years = len(returns) / 252
    annual_return = (1 + total_return) ** (1/years) - 1
    sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
    
    industry_stats.append({
        'code': code,
        'name': name,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'max_drawdown': max_drawdown,
        'sharpe': sharpe,
        'avg_pe': data['pe'].mean(),
        'avg_pb': data['pb'].mean(),
    })

industry_df = pd.DataFrame(industry_stats).sort_values('total_return', ascending=False)
industry_df.set_index('name', inplace=True)
industry_df

In [ ]:
# 可视化：行业收益率排名
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 总收益率
ax1 = axes[0, 0]
colors = ['green' if x > 0 else 'red' for x in industry_df['total_return']]
industry_df['total_return'].plot(kind='barh', ax=ax1, color=colors)
ax1.set_title('行业ETF总收益率 (2022-2026)')
ax1.set_xlabel('收益率')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))

# 2. 风险收益散点图
ax2 = axes[0, 1]
scatter = ax2.scatter(industry_df['volatility'], industry_df['total_return'], 
                      c=industry_df['sharpe'], cmap='RdYlGn', s=100)
for idx, row in industry_df.iterrows():
    ax2.annotate(idx, (row['volatility'], row['total_return']), fontsize=8)
ax2.set_xlabel('年化波动率')
ax2.set_ylabel('总收益率')
ax2.set_title('行业风险收益分布')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
plt.colorbar(scatter, ax=ax2, label='夏普比率')

# 3. 最大回撤
ax3 = axes[1, 0]
industry_df['max_drawdown'].sort_values().plot(kind='barh', ax=ax3, color='red', alpha=0.7)
ax3.set_title('行业ETF最大回撤')
ax3.set_xlabel('最大回撤')
ax3.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))

# 4. 估值分布
ax4 = axes[1, 1]
ax4.scatter(industry_df['avg_pe'], industry_df['avg_pb'], s=100, alpha=0.7)
for idx, row in industry_df.iterrows():
    ax4.annotate(idx, (row['avg_pe'], row['avg_pb']), fontsize=8)
ax4.set_xlabel('平均PE')
ax4.set_ylabel('平均PB')
ax4.set_title('行业估值分布')

plt.tight_layout()
plt.savefig('/Users/allen/workspace/python/stock/Tushare-DuckDB/reports/research/industry_analysis.png', dpi=150)
plt.show()

## 三、宽基ETF分析

In [ ]:
# 计算宽基指数表现
broad_stats = []

for code, name in BROAD_INDICES.items():
    data = broad_data[broad_data['ts_code'] == code].copy()
    if len(data) < 100:
        continue
    
    data = data.sort_values('trade_date')
    total_return = data['close'].iloc[-1] / data['close'].iloc[0] - 1
    returns = data['close'].pct_change().dropna()
    volatility = returns.std() * np.sqrt(252)
    
    cummax = data['close'].cummax()
    drawdown = (data['close'] - cummax) / cummax
    max_drawdown = drawdown.min()
    
    years = len(returns) / 252
    annual_return = (1 + total_return) ** (1/years) - 1
    sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
    
    broad_stats.append({
        'code': code,
        'name': name,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'max_drawdown': max_drawdown,
        'sharpe': sharpe,
    })

broad_df = pd.DataFrame(broad_stats).sort_values('total_return', ascending=False)
broad_df.set_index('name', inplace=True)

print("宽基指数表现汇总:")
broad_df

In [ ]:
# 宽基指数走势对比
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 归一化走势
ax1 = axes[0]
for code, name in BROAD_INDICES.items():
    data = broad_data[broad_data['ts_code'] == code].copy()
    if len(data) < 100:
        continue
    data = data.sort_values('trade_date')
    normalized = data['close'] / data['close'].iloc[0]
    ax1.plot(data['trade_date'], normalized, label=name, linewidth=1.5)

ax1.set_title('宽基指数走势对比 (2022-2026)')
ax1.set_xlabel('日期')
ax1.set_ylabel('归一化净值')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 收益率柱状图
ax2 = axes[1]
colors = ['green' if x > 0 else 'red' for x in broad_df['total_return']]
broad_df['total_return'].plot(kind='bar', ax=ax2, color=colors)
ax2.set_title('宽基指数总收益率')
ax2.set_ylabel('收益率')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('/Users/allen/workspace/python/stock/Tushare-DuckDB/reports/research/broad_index_analysis.png', dpi=150)
plt.show()

## 四、轮动策略实现与回测

In [ ]:
# 准备价格数据（宽表格式）
prices = industry_data.pivot(index='trade_date', columns='ts_code', values='close')
prices = prices.sort_index()

# PE和PB数据
pe_data = industry_data.pivot(index='trade_date', columns='ts_code', values='pe')
pb_data = industry_data.pivot(index='trade_date', columns='ts_code', values='pb')

print(f"价格数据形状: {prices.shape}")
print(f"日期范围: {prices.index[0]} - {prices.index[-1]}")
prices.head()

In [ ]:
def momentum_rotation(prices, lookback=20, hold_period=20, top_n=3, transaction_cost=0.001):
    """动量轮动策略"""
    momentum = prices.pct_change(lookback)
    
    dates = prices.index[lookback:]
    portfolio_value = [1.0]
    holdings = {}
    rebalance_dates = []
    last_rebalance = None
    turnover_list = []
    
    for i, date in enumerate(dates):
        if i == 0:
            mom_scores = momentum.loc[date].dropna()
            if len(mom_scores) < top_n:
                portfolio_value.append(portfolio_value[-1])
                continue
            
            top_assets = mom_scores.nlargest(top_n).index.tolist()
            holdings = {asset: 1.0/top_n for asset in top_assets}
            last_rebalance = date
            rebalance_dates.append(date)
            
            cost = transaction_cost * 2
            portfolio_value.append(portfolio_value[-1] * (1 - cost))
            continue
        
        # 计算当日收益
        daily_return = 0
        for asset, weight in holdings.items():
            if asset in prices.columns:
                prev_price = prices.loc[dates[i-1], asset]
                curr_price = prices.loc[date, asset]
                if pd.notna(prev_price) and pd.notna(curr_price) and prev_price > 0:
                    daily_return += weight * (curr_price / prev_price - 1)
        
        new_value = portfolio_value[-1] * (1 + daily_return)
        
        # 检查是否需要调仓
        days_since_rebalance = (date - last_rebalance).days
        if days_since_rebalance >= hold_period:
            mom_scores = momentum.loc[date].dropna()
            if len(mom_scores) >= top_n:
                new_top_assets = mom_scores.nlargest(top_n).index.tolist()
                
                old_assets = set(holdings.keys())
                new_assets = set(new_top_assets)
                turnover = len(old_assets - new_assets) / top_n
                turnover_list.append(turnover)
                
                cost = transaction_cost * 2 * turnover
                new_value = new_value * (1 - cost)
                
                holdings = {asset: 1.0/top_n for asset in new_top_assets}
                last_rebalance = date
                rebalance_dates.append(date)
        
        portfolio_value.append(new_value)
    
    return pd.Series(portfolio_value[1:], index=dates), np.mean(turnover_list) if turnover_list else 0

In [ ]:
def mean_reversion_rotation(prices, lookback=20, hold_period=20, top_n=3, transaction_cost=0.001):
    """均值回归轮动策略"""
    # 计算均值回归得分
    ma_short = prices.rolling(window=5).mean()
    ma_long = prices.rolling(window=lookback).mean()
    deviation = (ma_short - ma_long) / ma_long
    mr_score = -deviation  # 取负值，偏离越大（价格越低），得分越高
    
    # 计算RSI
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = (-delta).where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    start_idx = max(lookback, 14)
    dates = prices.index[start_idx:]
    portfolio_value = [1.0]
    holdings = {}
    last_rebalance = None
    turnover_list = []
    
    for i, date in enumerate(dates):
        if i == 0:
            mr_today = mr_score.loc[date].dropna()
            rsi_today = rsi.loc[date].dropna()
            
            combined_score = mr_today.copy()
            for asset in combined_score.index:
                if asset in rsi_today.index and rsi_today[asset] > 50:
                    combined_score[asset] = -999
            
            if len(combined_score[combined_score > -999]) < top_n:
                portfolio_value.append(portfolio_value[-1])
                continue
            
            top_assets = combined_score.nlargest(top_n).index.tolist()
            holdings = {asset: 1.0/top_n for asset in top_assets}
            last_rebalance = date
            
            cost = transaction_cost * 2
            portfolio_value.append(portfolio_value[-1] * (1 - cost))
            continue
        
        daily_return = 0
        for asset, weight in holdings.items():
            if asset in prices.columns:
                prev_price = prices.loc[dates[i-1], asset]
                curr_price = prices.loc[date, asset]
                if pd.notna(prev_price) and pd.notna(curr_price) and prev_price > 0:
                    daily_return += weight * (curr_price / prev_price - 1)
        
        new_value = portfolio_value[-1] * (1 + daily_return)
        
        days_since_rebalance = (date - last_rebalance).days
        if days_since_rebalance >= hold_period:
            mr_today = mr_score.loc[date].dropna()
            rsi_today = rsi.loc[date].dropna()
            
            combined_score = mr_today.copy()
            for asset in combined_score.index:
                if asset in rsi_today.index and rsi_today[asset] > 50:
                    combined_score[asset] = -999
            
            if len(combined_score[combined_score > -999]) >= top_n:
                new_top_assets = combined_score.nlargest(top_n).index.tolist()
                
                old_assets = set(holdings.keys())
                new_assets = set(new_top_assets)
                turnover = len(old_assets - new_assets) / top_n
                turnover_list.append(turnover)
                
                cost = transaction_cost * 2 * turnover
                new_value = new_value * (1 - cost)
                
                holdings = {asset: 1.0/top_n for asset in new_top_assets}
                last_rebalance = date
        
        portfolio_value.append(new_value)
    
    return pd.Series(portfolio_value[1:], index=dates), np.mean(turnover_list) if turnover_list else 0

In [ ]:
def multi_factor_rotation(prices, pe_data=None, pb_data=None, lookback=20, hold_period=20, top_n=3, transaction_cost=0.001):
    """多因子轮动策略"""
    # 计算各因子
    momentum = prices.pct_change(lookback)
    volatility = prices.pct_change().rolling(window=lookback).std() * np.sqrt(252)
    
    ma_short = prices.rolling(window=5).mean()
    ma_long = prices.rolling(window=lookback).mean()
    mr_score = -(ma_short - ma_long) / ma_long
    
    # 截面排名
    def cross_sectional_rank(df):
        return df.rank(axis=1, pct=True)
    
    mom_rank = cross_sectional_rank(momentum)
    vol_rank = 1 - cross_sectional_rank(volatility)  # 低波动率得分高
    mr_rank = cross_sectional_rank(mr_score)
    
    # 估值因子
    if pe_data is not None:
        pe_rank = 1 - cross_sectional_rank(pe_data)
    else:
        pe_rank = pd.DataFrame(0.5, index=prices.index, columns=prices.columns)
    
    if pb_data is not None:
        pb_rank = 1 - cross_sectional_rank(pb_data)
    else:
        pb_rank = pd.DataFrame(0.5, index=prices.index, columns=prices.columns)
    
    val_rank = (pe_rank + pb_rank) / 2
    
    # 综合得分
    composite_score = 0.4 * mom_rank + 0.2 * vol_rank + 0.2 * val_rank + 0.2 * mr_rank
    
    dates = prices.index[lookback:]
    portfolio_value = [1.0]
    holdings = {}
    last_rebalance = None
    turnover_list = []
    
    for i, date in enumerate(dates):
        if i == 0:
            scores = composite_score.loc[date].dropna()
            if len(scores) < top_n:
                portfolio_value.append(portfolio_value[-1])
                continue
            
            top_assets = scores.nlargest(top_n).index.tolist()
            holdings = {asset: 1.0/top_n for asset in top_assets}
            last_rebalance = date
            
            cost = transaction_cost * 2
            portfolio_value.append(portfolio_value[-1] * (1 - cost))
            continue
        
        daily_return = 0
        for asset, weight in holdings.items():
            if asset in prices.columns:
                prev_price = prices.loc[dates[i-1], asset]
                curr_price = prices.loc[date, asset]
                if pd.notna(prev_price) and pd.notna(curr_price) and prev_price > 0:
                    daily_return += weight * (curr_price / prev_price - 1)
        
        new_value = portfolio_value[-1] * (1 + daily_return)
        
        days_since_rebalance = (date - last_rebalance).days
        if days_since_rebalance >= hold_period:
            scores = composite_score.loc[date].dropna()
            if len(scores) >= top_n:
                new_top_assets = scores.nlargest(top_n).index.tolist()
                
                old_assets = set(holdings.keys())
                new_assets = set(new_top_assets)
                turnover = len(old_assets - new_assets) / top_n
                turnover_list.append(turnover)
                
                cost = transaction_cost * 2 * turnover
                new_value = new_value * (1 - cost)
                
                holdings = {asset: 1.0/top_n for asset in new_top_assets}
                last_rebalance = date
        
        portfolio_value.append(new_value)
    
    return pd.Series(portfolio_value[1:], index=dates), np.mean(turnover_list) if turnover_list else 0

In [ ]:
def calculate_metrics(pv):
    """计算绩效指标"""
    returns = pv.pct_change().dropna()
    total_return = pv.iloc[-1] / pv.iloc[0] - 1
    years = len(returns) / 252
    annual_return = (1 + total_return) ** (1/years) - 1
    volatility = returns.std() * np.sqrt(252)
    sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
    
    cummax = pv.cummax()
    drawdown = (pv - cummax) / cummax
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe': sharpe,
        'max_drawdown': max_drawdown,
    }

In [ ]:
# 运行不同周期的回测
periods = [10, 20, 40, 60]
results = {}

for period in periods:
    print(f"回测周期: {period}天...")
    
    mom_pv, mom_turnover = momentum_rotation(prices, lookback=period, hold_period=period)
    mr_pv, mr_turnover = mean_reversion_rotation(prices, lookback=period, hold_period=period)
    mf_pv, mf_turnover = multi_factor_rotation(prices, pe_data, pb_data, lookback=period, hold_period=period)
    
    results[period] = {
        'momentum': {'pv': mom_pv, 'turnover': mom_turnover, **calculate_metrics(mom_pv)},
        'mean_reversion': {'pv': mr_pv, 'turnover': mr_turnover, **calculate_metrics(mr_pv)},
        'multi_factor': {'pv': mf_pv, 'turnover': mf_turnover, **calculate_metrics(mf_pv)},
    }

print("回测完成!")

In [ ]:
# 准备基准数据（沪深300）
hs300 = broad_data[broad_data['ts_code'] == '000300.SH'].copy()
hs300 = hs300.set_index('trade_date')['close']
hs300_normalized = hs300 / hs300.iloc[0]

In [ ]:
# 可视化策略对比
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = {'momentum': 'blue', 'mean_reversion': 'green', 'multi_factor': 'red'}
labels = {'momentum': '动量策略', 'mean_reversion': '均值回归', 'multi_factor': '多因子'}

for idx, period in enumerate(periods):
    ax = axes[idx // 2, idx % 2]
    
    for strat_name in ['momentum', 'mean_reversion', 'multi_factor']:
        pv = results[period][strat_name]['pv']
        ax.plot(pv.index, pv.values, color=colors[strat_name], label=labels[strat_name], linewidth=1.5)
    
    # 添加基准
    bench = hs300_normalized.reindex(results[period]['momentum']['pv'].index)
    ax.plot(bench.index, bench.values, color='gray', linestyle='--', label='沪深300', linewidth=1)
    
    ax.set_title(f'持仓周期: {period}天')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('净值')

plt.tight_layout()
plt.savefig('/Users/allen/workspace/python/stock/Tushare-DuckDB/reports/research/strategy_comparison.png', dpi=150)
plt.show()

## 五、策略绩效汇总

In [ ]:
# 汇总所有策略表现
summary_data = []

for period in periods:
    for strat_name in ['momentum', 'mean_reversion', 'multi_factor']:
        r = results[period][strat_name]
        summary_data.append({
            '周期': f'{period}天',
            '策略': labels[strat_name],
            '总收益率': f"{r['total_return']:.2%}",
            '年化收益': f"{r['annual_return']:.2%}",
            '年化波动': f"{r['volatility']:.2%}",
            '夏普比率': f"{r['sharpe']:.2f}",
            '最大回撤': f"{r['max_drawdown']:.2%}",
            '平均换手': f"{r['turnover']:.2%}",
        })

summary_df = pd.DataFrame(summary_data)
print("策略绩效汇总表:")
summary_df

In [ ]:
# 交易成本敏感性分析
cost_levels = [0.0, 0.001, 0.002, 0.003, 0.005]
cost_results = []

for cost in cost_levels:
    mom_pv, _ = momentum_rotation(prices, lookback=20, hold_period=20, transaction_cost=cost)
    mr_pv, _ = mean_reversion_rotation(prices, lookback=20, hold_period=20, transaction_cost=cost)
    mf_pv, _ = multi_factor_rotation(prices, pe_data, pb_data, lookback=20, hold_period=20, transaction_cost=cost)
    
    cost_results.append({
        'cost': cost,
        'momentum': mom_pv.iloc[-1] / mom_pv.iloc[0] - 1,
        'mean_reversion': mr_pv.iloc[-1] / mr_pv.iloc[0] - 1,
        'multi_factor': mf_pv.iloc[-1] / mf_pv.iloc[0] - 1,
    })

cost_df = pd.DataFrame(cost_results)
cost_df.set_index('cost', inplace=True)

# 可视化
fig, ax = plt.subplots(figsize=(10, 6))
cost_df.plot(ax=ax, marker='o')
ax.set_title('交易成本对策略收益的影响 (20天周期)')
ax.set_xlabel('单边交易成本')
ax.set_ylabel('总收益率')
ax.legend(['动量策略', '均值回归', '多因子'])
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1%}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.savefig('/Users/allen/workspace/python/stock/Tushare-DuckDB/reports/research/cost_sensitivity.png', dpi=150)
plt.show()

print("\n交易成本敏感性分析:")
cost_df

## 六、结论与建议

In [ ]:
# 找出最优策略
best_results = []
for period in periods:
    for strat_name in ['momentum', 'mean_reversion', 'multi_factor']:
        r = results[period][strat_name]
        best_results.append({
            'period': period,
            'strategy': strat_name,
            'return': r['total_return'],
            'sharpe': r['sharpe'],
        })

best_by_return = max(best_results, key=lambda x: x['return'])
best_by_sharpe = max(best_results, key=lambda x: x['sharpe'])

print("="*60)
print("研究结论")
print("="*60)
print(f"\n1. 最高收益策略: {labels[best_by_return['strategy']]} (周期{best_by_return['period']}天)")
print(f"   - 总收益率: {best_by_return['return']:.2%}")
print(f"\n2. 最高夏普比率策略: {labels[best_by_sharpe['strategy']]} (周期{best_by_sharpe['period']}天)")
print(f"   - 夏普比率: {best_by_sharpe['sharpe']:.2f}")
print("\n3. 策略特点总结:")
print("   - 动量策略: 趋势市场表现好，但震荡市场容易追高")
print("   - 均值回归策略: 震荡市场表现好，但趋势市场容易抄底被套")
print("   - 多因子策略: 综合表现稳定，风险收益比较平衡")
print("\n4. 实操建议:")
print("   - 建议使用40-60天的调仓周期，可以降低交易成本")
print("   - 在牛市中可侧重动量策略")
print("   - 在震荡市中可侧重均值回归策略")
print("   - 注意控制交易成本，0.1%的单边成本会显著影响收益")

In [ ]:
# 关闭数据库连接
conn.close()
print("数据库连接已关闭")